In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from datasets import load_dataset

# Load slang dataset
slang_ds = load_dataset("MLBtrio/genz-slang-dataset")

README.md: 0.00B [00:00, ?B/s]

all_slangs.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1779 [00:00<?, ? examples/s]

In [9]:
slang_dict = {row["Slang"]: row["Description"] for row in slang_ds["train"]}

In [15]:
from transformers import pipeline

# Speech-to-text pipeline
asr = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-large-960h")

audio_file = "/content/audio4.wav"
transcription = asr(audio_file)
text_from_audio = transcription['text']

print("Transcribed text:", text_from_audio)


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

Device set to use cpu


Transcribed text: HALLO I GO TO SENTUINS AN POLOTIC COLGE OF IGYING AT EGNOGY AND I JUST  WONDER WHAT THE HELD THAT COLLEGY IS LIKE THAT COLLEGE EATUS LIKE OUR CHILLIANS YOU CAN'T GO OUT OF THE CLASS AND THEN THAT HOLE FOOLISH FACULTY EVERYWHERE IKE THE JUST ACT LIKE THERE WE'RE MORE MUCH OL THAN US LIKE THE JUST ACT LIKE CHILLENS I LIKE WHY DON'T TI COME BET YOUR ASSERMENS AND WORD HY DON'T  DO YOU OWN WORK LIKE WHAT ARE WE TWEKVE YEARS OLD OT THE WOR O SHIT FOR AND THEN THEY JUST TALK ABOUT SOME ANYPE HERE A NEW EDUCATION OF POLICY AND ALL THE BIGGEST BULL SHET EVER LIKE YOU CAN'T LET US GO OUT OF THE CLASS YOU CAN'T LET US DO OR ANYTHING THATIS AT IS E ARE APART FROM OUR CYLABRES WEN YOU CAN'T DO CAN'T LET US DO ANYTHING IN YOUR SLABURS DO YOU CAN'T LET US PERFORM ANY PROLJECT AND WHATEVER AND THEN YOU TALK OAT I NOISION SOME SOME CAR I SOME EXTRACALCOLD ACTIVITY SOME CREAT DUTY WERE WHAT IS CREAT ITY YOUR COLLEGES THE BIGGEST SHEET I EVER SEEN ITS US BEGATI O E THANK YOU PLEASE DON

In [16]:
# slang normalization
def normalize_slang(text, slang_dict):
    import re
    for slang, meaning in slang_dict.items():
        text = re.sub(rf"\b{re.escape(slang)}\b", meaning, text, flags=re.IGNORECASE)
    return text

normalized_text = normalize_slang(text_from_audio, slang_dict)
print("Normalized text:", normalized_text)


Normalized text: HALLO I GO TO SENTUINS AN POLOTIC COLGE OF IGYING At your terminal EGNOGY AND I JUST  WONDER WHAT THE HELD THAT COLLEGY IS LIKE THAT COLLEGE EATUS LIKE OUR CHILLIANS YOU CAN'T GO OUT OF THE CLASS AND THEN THAT HOLE FOOLISH FACULTY EVERYWHERE IKE THE JUST ACT LIKE THERE Whatever'RE MORE MUCH Old lady THAN US LIKE THE JUST ACT LIKE CHILLENS I LIKE WHY DON'T TI COME Yes, ok, "it's on." YOUR ASSERMENS AND Slang for "okay." HY DON'T  DO YOU OWN WORK LIKE WHAT Acronym rich environment Whatever TWEKVE YEARS OLD Off topic  THE WOR Over SHIT FOR AND THEN THEY JUST TALK ABOUT SOME ANYPE HERE A NEW EDUCATION OF POLICY AND ALL THE BIGGEST BULL SHET EVER LIKE YOU CAN'T LET US GO OUT OF THE CLASS YOU CAN'T LET US DO OR ANYTHING THATIS At your terminal IS Enemy Acronym rich environment APART FROM OUR CYLABRES WEN YOU CAN'T DO CAN'T LET US DO ANYTHING IN YOUR SLABURS DO YOU CAN'T LET US PERFORM ANY PROLJECT AND WHATEVER AND THEN YOU TALK OAT I NOISION SOME SOME CAR I SOME EXTRACALCOLD

In [17]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

save_dir = "./genz_emotion_model"

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/genz_emotion_model")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/genz_emotion_model")
text_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
# Predict emotion from transcribed text
emotion_result = text_classifier(normalized_text)
print("Emotion from speech:", emotion_result)


Device set to use cpu


Emotion from speech: [{'label': 'anger', 'score': 0.46212172508239746}]


In [18]:
from transformers import pipeline


audio_classifier = pipeline("audio-classification", model="superb/hubert-large-superb-er")
audio_result = audio_classifier(audio_file)
print("Audio tone result:", audio_result)


Device set to use cpu


Audio tone result: [{'score': 0.5933418869972229, 'label': 'hap'}, {'score': 0.23220910131931305, 'label': 'neu'}, {'score': 0.17043037712574005, 'label': 'sad'}, {'score': 0.004018626641482115, 'label': 'ang'}]


In [20]:
# Example mapping from audio labels to your text emotion labels
audio_to_text_map = {
    "happy": "joy",
    "sad": "sadness",
    "angry": "anger",
    "fearful": "fear",
    "disgusted": "disgust",
    "neutral": "neutral",
    "surprised": "surprise",
    "excited": "excitement",
    # add more as needed
}


In [21]:
def fuse_text_audio_confidence(text_res, audio_res, audio_map, text_weight=0.6):
    """
    Fuse text + audio emotion predictions with confidence scores.

    text_res: output from text classifier
    audio_res: output from audio classifier
    audio_map: dict to map audio labels to text labels
    text_weight: weight for text confidence (0–1)
    """
    # Extract text prediction
    text_label = text_res[0]['label']
    text_score = text_res[0]['score']

    # Extract audio prediction
    raw_audio_label = audio_res[0]['label'].lower()
    audio_score = audio_res[0]['score']

    # Map audio label to text emotion label
    audio_label = audio_map.get(raw_audio_label, "neutral")

    # Compute weighted scores
    weighted_text = text_score * text_weight
    weighted_audio = audio_score * (1 - text_weight)

    # Choose label with higher weighted score
    if weighted_text >= weighted_audio:
        final_label = text_label
        final_score = weighted_text
    else:
        final_label = audio_label
        final_score = weighted_audio

    return {"label": final_label, "score": final_score}


In [23]:
final_emotion = fuse_text_audio_confidence(emotion_result, audio_result, audio_to_text_map, text_weight=0.6)
print("Final emotion:", final_emotion)


Final emotion: {'label': 'anger', 'score': 0.27727303504943845}
